In [1]:
import numpy as np
import random
import math

In [2]:
class Node:
    def __init__(self, state, parent=None, action=None):
        self.state = state  # (belief, seeker_position)
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0
        self.actions = self.get_possible_actions()
        self.action_taken = action  # Store the action that led to this node

    def get_possible_actions(self):
        """Returns a list of valid actions (velocity commands)"""
        return ['FORWARD', 'LEFT', 'RIGHT']  # Example drone actions

    def is_fully_expanded(self):
        """Check if all actions have been tried from this node"""
        return len(self.children) == len(self.actions)

    def best_child(self, exploration_weight=1.0):
        """
        Selects the best child node using UCT (Upper Confidence Bound for Trees).
        
        UCT Formula:
            UCB = (Q(s, a) / N(s, a)) + C * sqrt(log(N(s)) / N(s, a))
        
        Where:
            - Q(s, a) is the total value of the node (total reward).
            - N(s, a) is the number of times the node was visited.
            - N(s) is the number of times the parent was visited.
            - C (exploration_weight) controls exploration vs. exploitation.
        
        Returns:
            The child node with the best UCT value.
        """
        best_node = None
        best_uct_value = float('-inf')
    
        for child in self.children:
            # Exploitation term: mean reward of this action
            exploitation = child.value / (child.visits + 1e-6)  # Avoid division by zero
            
            # Exploration term: encourages trying less-visited nodes
            exploration = exploration_weight * math.sqrt(math.log(self.visits + 1) / (child.visits + 1e-6))
            
            # Compute total UCT value
            uct_value = exploitation + exploration
    
            # Keep track of the best child
            if uct_value > best_uct_value:
                best_uct_value = uct_value
                best_node = child
    
        return best_node  # Return the child with the highest UCT score
        
    def get_untried_actions(self):
        """Returns actions that have not been explored yet from this node."""
        tried_actions = [child.action_taken for child in self.children]  # Track actions
        return [a for a in self.actions if a not in tried_actions]

    def add_child(self, new_state, action):
        """Adds a new child node corresponding to the given action."""
        child = Node(new_state, parent=self, action=action)
        self.children.append(child)
        return child

In [3]:
def mcts_search(initial_state, simulations=1000, horizon=10, exploration_weight=1.0):
    """
    Runs MCTS to find the best action for the seeker drone.
    Parameters:
        initial_state (tuple): (belief, seeker_position)
        simulations (int): Number of MCTS iterations
        horizon (int): Number of steps in each rollout
        exploration_weight (float): UCT exploration parameter
    Returns:
        best_action (str): The best action to take from the root state
    """
    
    root = Node(initial_state)

    for _ in range(simulations):
        # Step 1: Selection
        node = root
        while node.is_fully_expanded() and node.children:
            node = node.best_child(exploration_weight)

        # Step 2: Expansion
        untried_actions = node.get_untried_actions()
        if untried_actions:
            action = random.choice(untried_actions)
            new_state = simulate_action(node.state, action)  # Move to next state
            node = node.add_child(new_state, action)  # Add new child node

        # Step 3: Simulation (Rollout)
        total_cost = rollout(node.state, horizon)  

        # Step 4: Backpropagation
        while node:
            node.visits += 1
            node.value += total_cost  # Lower cost is better
            node = node.parent

    # Select best action from root node
    best_action = root.best_child(exploration_weight=0).state[1]  # Best action has no exploration factor
    return best_action

In [4]:
def simulate_action(state, action):
    """
    Simulates taking an action from a given state.
    Returns the new state (updated belief, new seeker position).
    """
    belief, seeker_pos = state
    new_seeker_pos = update_seeker_position(seeker_pos, action)
    new_belief = update_belief(belief, new_seeker_pos)  # Use particle filter update
    return (new_belief, new_seeker_pos)

def rollout(state, horizon):
    """
    Simulates a sequence of random actions to estimate the cost.
    Uses a simple policy (random actions).
    """
    total_cost = 0
    belief, seeker_pos = state
    for _ in range(horizon):
        action = random.choice(['FORWARD', 'LEFT', 'RIGHT'])
        seeker_pos = update_seeker_position(seeker_pos, action)
        belief = update_belief(belief, seeker_pos)  # Particle filter update
        total_cost += cost_function(belief, seeker_pos)
    return total_cost

def update_seeker_position(seeker_pos, action):
    """
    Updates the seeker drone's position based on the selected action.
    """
    if action == 'FORWARD':
        return seeker_pos + np.array([1, 0])  # Move forward
    elif action == 'LEFT':
        return seeker_pos + np.array([0, -1])  # Turn left
    elif action == 'RIGHT':
        return seeker_pos + np.array([0, 1])  # Turn right
    return seeker_pos

def update_belief(belief, seeker_pos):
    """
    Updates the particle filter (belief) based on the new seeker position.
    """
    return particle_filter_update(belief, seeker_pos)  # Assume existing function

def cost_function(belief, seeker_pos):
    """
    Computes the cost function c(s_t) = H(b_t) + λ E[1(collision)].
    """
    entropy = compute_entropy(belief)
    collision_penalty = compute_collision_penalty(belief, seeker_pos)
    lambda_weight = 10  # Tunable parameter
    return entropy + lambda_weight * collision_penalty
